In [17]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()



In [19]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot


In [20]:
import torch

In [21]:

x1 = torch.Tensor([2.0]).double()                ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()                ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()               ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()                ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double()  ; b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

0.7071066904050358
---
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [81]:

class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    #print('Neurons')
    #print(self.w)
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    #print('wi*xi')
    #print(list(zip(self.w, x)))
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    # sz defines num inputs and num outputs at each layer
    sz = [nin] + nouts
    #print('sz')
    #print(sz)
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]


In [82]:
# single neuron with 2 inputs
x = [2.0, 3.0]
n = Neuron(2)
n(x)

Value(data=-0.9919581672804689)

In [83]:
# single layer with 2 inputs and 3 neurons
x = [2.0, 3.0]
n = Layer(2, 3)
n(x)

[Value(data=0.24379008865581456),
 Value(data=-0.6304124126735503),
 Value(data=-0.9659965060383164)]

In [122]:
# MLP with 2 inputs and 3 layers, with 4, 4, and 1 neurons respectively. used in example below
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=0.9913932762441241)

In [123]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [85]:
ypred = [n(x) for x in xs]
loss = sum([(yout-ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

Value(data=6.637440582619711)

In [106]:
for p in n.parameters():
    # remember to flush grad
    p.grad = 0.0
loss.backward()
print(n.layers[0].neurons[0].w[0].grad)
print(n.layers[0].neurons[0].w[0].data)

0.017932494508883665
0.3167396935517021


In [100]:
for p in n.parameters():
    p.data+= -0.1 * p.grad

In [101]:
ypred = [n(x) for x in xs]
loss = sum([(yout-ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

Value(data=0.004169901996572124)

In [102]:
ypred

[Value(data=0.9992824851401281),
 Value(data=-0.9671445448031338),
 Value(data=-0.9444385061515762),
 Value(data=0.998318740312661)]

In [104]:
n.parameters()

[Value(data=0.3167396935517021),
 Value(data=0.39824518835813927),
 Value(data=0.15981502023486416),
 Value(data=0.6491925825247705),
 Value(data=-1.0677312441042865),
 Value(data=-0.49534534065123015),
 Value(data=0.5498869376611413),
 Value(data=-0.17093004618024743),
 Value(data=-0.22681401586347222),
 Value(data=-0.4324140843906282),
 Value(data=0.10414504984651296),
 Value(data=-0.5336896185771127),
 Value(data=-0.2973355579319776),
 Value(data=1.1043433238099272),
 Value(data=-1.1163838226199878),
 Value(data=-0.6985819860915974),
 Value(data=0.5097808670913105),
 Value(data=-0.5319739841515339),
 Value(data=-0.3284433331158072),
 Value(data=-0.3605485633917881),
 Value(data=1.4234789428411094),
 Value(data=0.09663588141386087),
 Value(data=0.37366972647545915),
 Value(data=0.03693679205602672),
 Value(data=0.9755893488852292),
 Value(data=-0.8592769510118607),
 Value(data=0.6843250476599277),
 Value(data=-0.6795779121024804),
 Value(data=0.8699333936931091),
 Value(data=0.475088

In [129]:

for k in range(20):
  
  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
  
  # backward pass
  for p in n.parameters():
    # remember to flush grad
    p.grad = 0.0
  loss.backward()
  
  # update
  for p in n.parameters():
    p.data += -0.05 * p.grad
  
  print(k, loss.data)
  

0 0.009434298477912235
1 0.009283512999636687
2 0.009137253800859885
3 0.008995324488140577
4 0.00885753971861969
5 0.008723724440563106
6 0.008593713195273764
7 0.008467349474693476
8 0.008344485129606224
9 0.008224979823876115
10 0.008108700530617135
11 0.007995521066602035
12 0.007885321661583695
13 0.0077779885595279236
14 0.007673413649045812
15 0.007571494120574074
16 0.007472132148082334
17 0.00737523459329462
18 0.007280712730597369
19 0.007188481990974037


In [130]:
ypred

[Value(data=0.9668186390523691),
 Value(data=-0.9820996695304308),
 Value(data=-0.9469236789611907),
 Value(data=0.9456864511313933)]